In [1]:
import pandas as pd
import numpy as np
import fasttext
from keras.preprocessing.text import Tokenizer
# from gensim.models.fasttext import FastText
import matplotlib.pyplot as plt
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk import WordPunctTokenizer

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
from bs4 import BeautifulSoup
import re
import emoji
import csv
import itertools

%matplotlib inline

Using TensorFlow backend.
[nltk_data] Downloading package punkt to C:\Users\Yasira
[nltk_data]     Younus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Yasira
[nltk_data]     Younus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Yasira
[nltk_data]     Younus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train_set = pd.read_csv('train.csv')
train_set.drop(['textID', 'selected_text'], axis=1, inplace=True)
train_set = train_set.dropna()
train_set.reset_index(drop=True, inplace=True)

test_set = pd.read_csv('test.csv')
test_set.drop(['textID'], axis=1, inplace=True)
test_set = test_set.dropna()
test_set.reset_index(drop=True, inplace=True)
test_set

dataset = pd.concat([train_set, test_set], axis=0)
dataset = dataset[dataset['sentiment'] != "neutral"]
dataset.reset_index(drop=True, inplace=True)
dataset

,text,sentiment
0,Sooo SAD I will miss you here in San Diego!!!,negative
1,my boss is bullying me...,negative
2,what interview! leave me alone,negative
3,"Sons of ****, why couldn`t they put them on t...",negative
4,2am feedings for the baby are fun when he is a...,positive
...,...,...
18462,"its at 3 am, im very tired but i can`t sleep ...",negative
18463,All alone in this old house again. Thanks for...,positive
18464,I know what you mean. My little dog is sinkin...,negative
18465,_sutra what is your next youtube video gonna b...,positive


In [3]:
def load_dict_smileys():
    
    return {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }

# self defined contractions
def load_dict_contractions():
    
    return {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "I'd":"I would",
        "I'll":"I will",
        "I'm":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "luv":"love",
        "sux":"sucks"
        }

In [5]:
train = open('tweets.train','w')  
test = open('tweets.valid','w')  

clean_tweets=[]
line = 0
for row in dataset.iterrows():
                  
    tweet = BeautifulSoup(row[1].text).get_text()
    tweet = tweet.replace('\x92',"'")
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", tweet).split())
    tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
    tweet = ' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", tweet).split())
    tweet = tweet.lower()
    
    CONTRACTIONS = load_dict_contractions()
    tweet = tweet.replace("’","'")
    words = tweet.split()
    reformed = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in words]
    tweet = " ".join(reformed)
    
    tweet = ''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet))
    
    SMILEY = load_dict_smileys()  
    words = tweet.split()
    reformed = [SMILEY[word] if word in SMILEY else word for word in words]
    tweet = " ".join(reformed)
    
    tweet = emoji.demojize(tweet)
    tweet = tweet.replace(":"," ")
    tweet = ' '.join(tweet.split())
    row[1].text = tweet
    format_string = "__label__" + row[1].sentiment.upper() + " " + tweet
    clean_tweets.append(format_string)
                  
    line = line + 1
    if line%16 == 0:
        print(format_string, file=test)
    else:
        print(format_string, file=train)  

In [6]:
clean_tweets = pd.DataFrame(clean_tweets)
dataset['formatted_tweets'] = clean_tweets
dataset

,text,sentiment,formatted_tweets
0,soo sad i will miss you here in san diego,negative,__label__NEGATIVE soo sad i will miss you here...
1,my boss is bullying me,negative,__label__NEGATIVE my boss is bullying me
2,what interview leave me alone,negative,__label__NEGATIVE what interview leave me alone
3,sons of ** why couldn`t they put them on the r...,negative,__label__NEGATIVE sons of ** why couldn`t they...
4,2am feedings for the baby are fun when he is a...,positive,__label__POSITIVE 2am feedings for the baby ar...
...,...,...,...
18462,its at 3 am im very tired but i can`t sleep bu...,negative,__label__NEGATIVE its at 3 am im very tired bu...
18463,all alone in this old house again thanks for t...,positive,__label__POSITIVE all alone in this old house ...
18464,i know what you mean my little dog is sinking ...,negative,__label__NEGATIVE i know what you mean my litt...
18465,_sutra what is your next youtube video going t...,positive,__label__POSITIVE _sutra what is your next you...


In [7]:
model = fasttext.train_supervised(input='tweets.train', autotuneValidationFile='tweets.valid')
result = model.test('tweets.train')
validation = model.test('tweets.valid')
        
text_line = "accuracy:" + str(result[1])  + ", validation:" + str(validation[1]) + '\n' 
print(text_line)

accuracy:0.9998842458617896, validation:0.8927637314734089



In [8]:
model.quantize(input='tweets.train', retrain=True)

validation = model.test('tweets.valid')
validation[1]

0.8901482127288579

In [ ]:
model.save_model("model_fasttext.ftz")